<a href="https://colab.research.google.com/github/jinsu1214/class2022spring/blob/main/huggingface%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

Image classification
고양이 이미즐 넣었을 때 이 이미지가 어느 카테고리에 들어가는지 구분해준다. 고양이 사진을 넣으면 이집트 고양이인지 호랑이 고양이인지 구분한다.  
e.g. https://huggingface.co/google/vit-base-patch16-224  
how to use가 있는지부터 확인을 해야 한다.

In [ ]:
!pip install transformers

In [4]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


전부 다 이해하지는 못하더라도 배운 내용 안에서 읽을 수 있어야 한다.  
첫 단락에서 필요한 것을 가져오고 두번째 단락에서 url을 가져와 image를 가져온다.  
from pretrained가 있는데 pretrained는 행렬이다. 즉 ai이다. 데이터로부터 잘 훈련된 돈으 가치가 있는 행렬이니 모델이다. 이름으로 추측할 수 있다. feature extraction을 할 수 있는 모델이다. 드리고 다음거는 classfication이 있기 때문에 extraction된 것을 classfication을 해주는 모델이구나를 알 수 있다. feature라는 것은 이미지가 숫자정보로 되어 있고 컬러면 3층 정보로 있고 흑백이면 1층 정보로만 있다는 것을 알 수 있다. 숫자정보를 다 쓰기에는 너무나 정보가 많기에 숫자들 속에서 압축적으로 쓸 수 있는 정보를 뽑아낼 수 있다. 예를 들어 어떤 사람을 classify 한다고 했을 때 정보라는 것이 무한대 숫자로 있지만 이것을 feature한다면 눈이 어떻고 옷이 어떤지 특정 부문을 제한해 정보를 뽑아내는 것이다.   
뽑아낸 숫자정보가 원래 원본 image보다 적다.  
우리가 뽑아낸 feature 정보로부터 1000개의 클래스가 준비되어 있고 각 클래스마다 확률정보가 계산되어 나온다. 1000개 마다 각 숫자가 배정되기에 다 더하면 1이 된다는 것을 알 수 있다. 이 중에서 확률이 가장 큰 것이 무엇인지를 찾는 것이 idx다.   
이는 함수의 형태가 아니고 script형태이다. 함수의 형태는 in&out이 있다.

Demo in Gradio

In [5]:
def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

image라는 입력을 넣었을 때 predicted class라는 output이 나오는 함수의 형ㅌ태로 변환하였다.

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()

입려그이 형태와 출력의 형태가 무엇인지 적어줘야 한다. 입력의 타입은 image고 출력의 타입은 텍스트이다. 입출력 유형을 적어줘야 한다.

Fill-Mask  
mask에 어떠한 단어가 들어갈지 단어마다 확률을 뽑아서 보여준다.  
e.g. https://huggingface.co/bert-base-uncased  


In [8]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

[{'score': 0.10731077939271927,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.0877445638179779,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338413640856743,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.046672213822603226,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095887809991837,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

모델 이름이 unmasker 즉 mask를 단어로 지워준다는 의미이다.  
이제 이것을 함수 형태 즉 gradio로 만들어야 한다.  
위의 결과를 보면 list로 dictionary가 5개 들어있다는 것을 확인할 수 있다. 따라서 데이터프레임하면 된다.

In [9]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  return df

In [11]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Token classification  
nlp중 하나이다. nayural language understanding. 어떤 natural language가 주어져 있다면 즉 한 문장이 주어져 있다면 자유도가 매우 높다. 컴퓨터는 모든 가능한 자유도를 계산하기 힘들다. 따라서 이것은 무엇을 말하는거다라고 뽑아낼 수 있어야 하고 이를 natural language inderstaning이다. ner은 어떤 문장이 주어져 있을 때 어떤 명사는 정보적으로 더 중요하다. 더욱 집중을 해야 하는 명사들이 있고 이러한 것들을 인식해 내는 것이 named entity regognition이라고 해서 ner이라고 말한다. 중요하게 생각되는 것은 장소, 조직, 사람 등의 정보이다.   
e.g. https://huggingface.co/dslim/bert-base-NER  


In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


tokenizer는 문장이 있으면 단어 단위로 쪼개는 것을 의미한다. 각각의 단어 즉 토큰들이 어느 class에 속하는지 인식해내는 것이 model.  
nlp가 모델이고 결과를 내는데 쓰인다.  
마찬가지로 이 결과도 dictionary의 list이기 때문에 dataframe으로 받는다.

In [14]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df

In [15]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Sentence similarity  
단어를 통해서도 알 수 있지만 주어진 두개의 문장이 있을 때 이들이 얼마나 유사한지를 보는 것이다. 모든 모델은 입력값은 텍스트를 그대로 받아들일 수 없다. raw데이터를 받아들일 수 없다. 따라서 feature의 형태로 변환해야 한다. 즉 embedding(vector)로 변환해야 한다. 바꿔주야지 두 텍스트 간의 유사성을 계산할 수 있다.  
e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2  


In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

모델을 가지고 입력으로 문장을 썼더니 embeddings를 산출한다. 하면 매우 많은 숫자들이 나온다. embeddings를 보면 [[  ],[  ]] 형식으로 되어 있다.  
len(embeddings[0])하면 0이든 1이든 다 384개가 나온다. 즉[ , , , , ,] 안에 384개의 숫자가 들어가 있는 것이다.  
예를 들어 우리한테 [1,2]라는 벡터가 있고 [2,1]이라는 두번쨰 벡터가 있다고 하자. 이를 [x,y]로 생각해 좌표평면에 점을 찍을 수 있다. 이 두개가 얼만큼 비슷한지는 원점과 점을 이었을 때의 각도로 이야기할 수 있다. 각도가 0도이면 유사도가 제일 높은 것이고 각도가 90도이면 유사도가 제일 낮다는 것을 의미한다. 근데 우리는 코사인을 붙인다. 이렇게 되면 cos에서 각도에 0이 들어가면 1이 되고 90이 들어가면 0이 된다. 즉 유사도가 제일 높으면 1이 나오소 유사도가 제일 낮으면 0이 된다. 각도에 코사인을 붙인다는 점을 꼭 기억하자.   
여기서는 embedding이 벡터이다. similarity는 각도의 코사인이기 떄문에 우리는 이것을 cosine similarity라고 부른다. 만약 숫자가 세개가 되면 3차원에서의 각도 4개면 4차원에서의 각도로 생가한다. 차원이 아무리 높아져도 원점과 함께 삼각형을 만들기 때문에 각도가 만들어진다. 이 각도에 코사인을 적용하면 -1에서 1사이의 값이 나올 것이다. 이 계산을 해주는 것이 아래 식이다.

In [19]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
cosine_scores

tensor([[0.5726]])

In [20]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [21]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()